# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm

### Load the bigrams data

In [6]:
bigrams = pd.read_csv('Assignment2_data/bigrams.txt', sep='\t', header=None, encoding='latin-1')

In [7]:
bigrams.head()

,0,1,2
0,275,a,a
1,31,a,aaa
2,29,a,all
3,45,a,an
4,192,a,and


### Create a dictionary of bigrams

In [8]:
# Convert to dictionary of (word1, word2): count
bigram_count = defaultdict(int, bigrams.set_index([1, 2]).to_dict()[0])
bigram_count_total = sum(bigram_count.values())

In [9]:
bigram_count

defaultdict(int,
            {('a', 'a'): 275,
             ('a', 'aaa'): 31,
             ('a', 'all'): 29,
             ('a', 'an'): 45,
             ('a', 'and'): 192,
             ('a', 'another'): 39,
             ('a', 'at'): 25,
             ('a', 'b'): 82,
             ('a', 'b+'): 45,
             ('a', 'b-17'): 26,
             ('a', 'b-2'): 31,
             ('a', 'b-52'): 54,
             ('a', 'b-movie'): 39,
             ('a', 'b-plus'): 40,
             ('a', 'b.a'): 168,
             ('a', 'b.f.a'): 64,
             ('a', 'b.s'): 84,
             ('a', 'ba'): 66,
             ('a', 'babble'): 41,
             ('a', 'babbling'): 28,
             ('a', 'babe'): 159,
             ('a', 'baboon'): 83,
             ('a', 'baby'): 9744,
             ('a', 'baby-faced'): 31,
             ('a', 'baby-sitter'): 122,
             ('a', 'babysitter'): 237,
             ('a', 'babysitting'): 23,
             ('a', 'baccalaureate'): 95,
             ('a', 'bach'): 71,
             ('

In [10]:
# Convert to dictionary of word: count
vocabulary = defaultdict(int)

for word1, word2 in bigram_count.keys():
    vocabulary[word1] += bigram_count[(word1, word2)]
    vocabulary[word2] += bigram_count[(word1, word2)]
    
vocabulary_total = sum(vocabulary.values())
print(vocabulary_total)

573516412


### Checker for known words

In [11]:
def known(words):
    """
    Method to get all known words from the input list of words
    :param words: list of str
    :return: set of str
    """
    return set(w for w in words if vocabulary[w] > 0)

### Get all words at a given Levenshtein distance

In [12]:
from functools import lru_cache

@lru_cache(maxsize=None)
def get_words_at_levenshtein_distance(word, distance):
    """
    Method to get all words at a given Levenshtein distance from the input word
    :param word: str
    :param distance: int
    :return: set of str
    """
    alphabet = 'abcdefghijklmnopqrstuvwxyz'

    if distance == 0:
        return {word}

    if distance == 1:
        splits      = [(word[:i], word[i:])     for i in range(len(word) + 1)]
        deletes     = [a + b[1:]                for a, b in splits if b]
        transposes  = [a + b[1] + b[0] + b[2:]  for a, b in splits if len(b) > 1]
        replaces    = [a + c + b[1:]            for a, b in splits if b for c in alphabet]
        inserts     = [a + c + b                for a, b in splits for c in alphabet]
        return set(deletes + transposes + replaces + inserts)
    
    else:
        output = set()
        for w in get_words_at_levenshtein_distance(word, distance - 1):
            output |= get_words_at_levenshtein_distance(w, 1)
        return output

In [13]:
print(get_words_at_levenshtein_distance('mth', 1))

{'mih', 'mta', 'hth', 'mzth', 'mthk', 'pth', 'sth', 'mthy', 'mtp', 'muh', 'mtwh', 'uth', 'mtha', 'mmh', 'mtx', 'eth', 'mnth', 'mtyh', 'mt', 'cmth', 'vmth', 'mdh', 'mts', 'mtbh', 'mtoh', 'mtn', 'mthw', 'mtjh', 'mtdh', 'rmth', 'mtgh', 'mtxh', 'mti', 'mtw', 'mtd', 'moh', 'mvh', 'mtl', 'mthh', 'jth', 'mthx', 'mtho', 'mah', 'mtz', 'mthg', 'mrh', 'emth', 'mwh', 'mthd', 'mthe', 'mtt', 'mjh', 'mtsh', 'mfh', 'mtrh', 'mxh', 'pmth', 'gth', 'gmth', 'wmth', 'imth', 'mgth', 'mtr', 'qmth', 'mtth', 'mthv', 'mths', 'tmth', 'mtu', 'lth', 'mzh', 'mthr', 'zmth', 'mthn', 'mtk', 'zth', 'xmth', 'mtvh', 'hmth', 'mthq', 'mth', 'kth', 'mch', 'mtlh', 'mtuh', 'umth', 'mtg', 'mtv', 'mto', 'mtb', 'mtm', 'mthf', 'mteh', 'mthm', 'mjth', 'mph', 'mcth', 'mtc', 'mfth', 'dmth', 'nth', 'nmth', 'dth', 'th', 'mbh', 'mtj', 'mthc', 'mthz', 'mtnh', 'jmth', 'mhth', 'qth', 'mgh', 'ath', 'oth', 'mmth', 'mqh', 'vth', 'rth', 'mlth', 'msh', 'cth', 'mvth', 'mkth', 'mwth', 'yth', 'math', 'omth', 'msth', 'mthj', 'mrth', 'mith', 'moth',

In [14]:
known(get_words_at_levenshtein_distance('keng', 1))

{'deng',
 'feng',
 'kang',
 'keg',
 'ken',
 'kent',
 'king',
 'kong',
 'kung',
 'peng',
 'seng'}

### With the above helper functions, we can now get set of candidates for a given word

In [15]:
def word_candidates(word, distance=1):
    candidates = known([word])

    n = 1
    while not candidates and n <= distance:
        candidates |= known(get_words_at_levenshtein_distance(word, n))
        n += 1

    if not candidates:
        return {word}

    return candidates

In [16]:
word_candidates('keng')

{'deng',
 'feng',
 'kang',
 'keg',
 'ken',
 'kent',
 'king',
 'kong',
 'kung',
 'peng',
 'seng'}

### Now we can get suggestions for a given word

In [17]:
def unigram_suggestions(word, edit_distance):
    """
    Method to get suggestions for the input word
    :param word: 1 word
    :param edit_distance: int 
    :return: suggestion
    """
    candidates = word_candidates(word, edit_distance)

    suggestions = []
    for word in candidates:
        prob = vocabulary[word] / vocabulary_total
        if prob > 0:
            suggestions.append((word, prob))

    return sorted(suggestions, key=lambda x: x[1], reverse=True)

In [18]:
unigram_suggestions('keng', 1)

[('king', 7.63238838228748e-05),
 ('kong', 2.1540795941511783e-05),
 ('ken', 1.570486879109573e-05),
 ('kent', 5.797567306583024e-06),
 ('deng', 2.2981033714515568e-06),
 ('kung', 1.229258631922115e-06),
 ('keg', 7.009389645853761e-07),
 ('feng', 6.817590426688609e-07),
 ('peng', 4.045219895119584e-07),
 ('seng', 2.80724311687178e-07),
 ('kang', 1.4472122900643337e-07)]

### Now we can get suggestions for a bigram

In [19]:
def bigram_suggestions(bigram, distance=1, left=True):
    candidates = word_candidates(bigram[0 if left else 1], distance)

    bigram_candidates = set()
    for candidate in candidates:
        if left:
            bigram_candidates.add((candidate, bigram[1]))
        else:
            bigram_candidates.add((bigram[0], candidate))

    suggestions = []
    for candidate in bigram_candidates:
        prob = bigram_count[candidate[0], candidate[1]] / vocabulary[word1] if vocabulary[word1] > 0 else 0
        if prob > 0:
            suggestions.append((candidate, prob))

    return sorted(suggestions, key=lambda x: x[1], reverse=True)

In [20]:
bigram_suggestions(('a', 'keng'), 1, left=False)

[(('a', 'king'), 17.041666666666668),
 (('a', 'keg'), 1.375),
 (('a', 'ken'), 0.5),
 (('a', 'kung'), 0.4583333333333333),
 (('a', 'feng'), 0.3888888888888889)]

### Now we can implement the autocorrect method

In [21]:
def autocorrect(sentence, distance = 1):
    """
    Method to correct the input sentence
    :param sentence: str
    :param distance: int 
    :return: str
    """
    words = sentence.split()

    if len(words) == 0:
        return []

    corrected = []

    for i in range(0, len(words)):
        # Get bigram suggestions for right and left
        r_sug = bigram_suggestions((words[i - 1], words[i]), distance, left=False) if i != 0 else []
        r_sug = [(w, p) for ((_, w), p) in r_sug]

        l_sug = bigram_suggestions((words[i], words[i + 1]), distance, left=True) if i != len(words) - 1 else []
        l_sug = [(w, p) for ((w, _), p) in l_sug]

        all_sug = l_sug + r_sug

        # if there are no suggestions, try unigram suggestions
        if len(all_sug) == 0 and not known([words[i]]):
            all_sug = unigram_suggestions(words[i], distance)

        # if still no suggestions, add the word as is
        if len(all_sug) == 0:
            corrected.append(words[i])
            continue

        # Pick the word by the max sum of unigram and bigram probabilities
        probs = defaultdict(float)
        for word, probability in all_sug:
            probs[word] += probability
            
        corrected.append(max(probs, key=probs.get)) if len(probs) > 0 else corrected.append(words[i])

    return " ".join(corrected)

In [22]:
autocorrect('Me wold like to go to the park but thera wus an acident on the way')

'he would like to go to the park but there was an accident on the way'

In [23]:
def norvig_correction(word: str, edit_distance: int = 1) -> str:
    candidates = word_candidates(word, edit_distance)
    # choose the best candidate
    return max(candidates, key=vocabulary.get)


def norvig_autocorrect(sentence: str, edit_distance: int = 1) -> str:
    sentence_corrected = []

    for word in sentence.split():
        sentence_corrected.append(norvig_correction(word, edit_distance))

    return " ".join(sentence_corrected)

In [24]:
norvig_autocorrect('Me wold like to go to the park but thera wus an acident on the way')

'he would like to go to the park but there was an accident on the way'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

For my implementation I used bigram dataset, which can obviously be limiting in correctness. It is still better than baseline for unigram, but sure is showing poorer performance as trigram. I did not try N-grams with n > 2, due to limitations in computational resources (it took kind of OK to process bigrams, but will a lot more for further sizes).

I used the same weights for edit1 and edit2, as I did not have a good reason to change them. I did not use any weights for absent words, as results that I got were quite alright without them. Maybe examples which are used were too bias, and implementing weights for absent words would be beneficial for other cases.



## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [25]:
# read txt file
with open('Assignment2_data/TwelfthNight.txt', 'r') as file:
    test_set = file.readlines()

In [27]:
test_set[:10]

['\n',
 'Twelfth Night\n',
 '\n',
 'ACT I\n',
 "SCENE I. DUKE ORSINO's palace.\n",
 'Enter DUKE ORSINO, CURIO, and other Lords; Musicians attending\n',
 'DUKE ORSINO\n',
 'If music be the food of love, play on;\n',
 'Give me excess of it, that, surfeiting,\n',
 'The appetite may sicken, and so die.\n']

In [30]:
# add noise to words (change or delete some letters
def add_noise_to_text(text, noise_probability):
    noised_text = []
    for line in text:
        noised_line = []
        for word in line.split():
            if np.random.rand() < noise_probability:
                noised_word = list(word)
                idx = np.random.randint(0, len(noised_word))
                noised_word[idx] = np.random.choice([c for c in 'abcdefghijklmnopqrstuvwxyz' if c != noised_word[idx]])
                noised_line.append(''.join(noised_word))
            else:
                noised_line.append(word)
        noised_text.append(' '.join(noised_line))
    return noised_text

In [31]:
noised_text = add_noise_to_text(test_set, 0.1)
noised_text[:10]

['',
 'Twxlfth Niuht',
 '',
 'ACT I',
 "SCENE I. DUKb ORSINO's palace.",
 'Enter DUKE ORmINO, CURIO, and other Lords; Musicians attending',
 'DUKE ORSINO',
 'If music be the food of love, play on;',
 'Give me eccess on ii, thate surfeiting,',
 'The appetite eay sickenh and so die.']

In [40]:
# evaluate the accuracy of the autocorrect method
def get_autocorrected_text(text, autocorrect_method):
    corrected_text = []
    for line in tqdm(text):
        corrected_text.append(autocorrect_method(line))
    return corrected_text

def evaluate_accuracy(corrected_text, original_text):
    correct = 0
    total = 0
    for corrected_line, original_line in zip(corrected_text, original_text):
        # if is not lsit of words, split it
        if not isinstance(corrected_line, list):
            
            corrected_words = corrected_line.split()
            original_words = original_line.split()
            for corrected_word, original_word in zip(corrected_words, original_words):
                if corrected_word == original_word:
                    correct += 1
                total += 1
    return correct / total

corrected_text = get_autocorrected_text(noised_text, autocorrect)
# change cirrected text to string
corrected_text = [line for line in corrected_text]

accuracy = evaluate_accuracy(corrected_text, test_set)
accuracy

100%|██████████| 3697/3697 [00:01<00:00, 2682.09it/s]


0.6837634913186298

In [41]:
def get_norvig_text(text):
    corrected_text = []
    for line in tqdm(text):
        corrected_text.append(norvig_autocorrect(line))
    return corrected_text

norvig_corrected_text = get_norvig_text(noised_text)
# change corrected text to string
norvig_corrected_text = [line for line in norvig_corrected_text]

norvig_accuracy = evaluate_accuracy(norvig_corrected_text, test_set)
norvig_accuracy

100%|██████████| 3697/3697 [00:00<00:00, 3778.51it/s]


0.6813233223838574

# Conclusion
The difference is obsolete between the two methods, as both have different accuracy .681 and .683.

I think the difference is not much, because of Shakespeare's text of Twelfth Night, which is quite old and has a lot of words that are not used today. This makes it hard for both methods to correct the text, as they are not trained on such old text (so bigrams).

Still results of my solution are better due to a more complex approach I propose.